# Handling missing data

The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing. To make matters even more complicated, different data sources may indicate missing data in different ways.

In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demosntrate some built-in Pandas tools for handling missing data in Python. Here and throughout the book, we'll refer to missing data in general as *null*, *NaN*, or *NA* values.

## Trade-Offs in Missing Data Conventions

There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: using a *mask* that globally indicates missing values, or choosing a *sentinel value* that indicates a missing entry.

In the masking approach, the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

In the sentinel approach, the sentinvel value could be some data-specific convention, such as indicating a missing einteger value with -9999 or some rare bit pattern, or it could be a more global convention, such as indicating a missing floating-point value with NaN, a special value which is part of the IEEE floating-point specification.

None of these approaches is without trade-offs: use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

As in most cases where no universally optimal choice exists, different languages and system use different conventions. For example, the R language uses reserved big patterns within each data type as sentinel values indicating missing data, while the SciDB system uses an extra byte attached to every cell which indicates a NA state.

## Missing Data in Pandas

The way in which Pandas handles missing values is contrained by its reliance on the NumPy package, which does not have a built-in notion of NA values for non-floating-point data types.

Pandas could have followed R's lead in specifying bit patterns for each indivisual data type to indicate nullness, but this approach turns out to be rather unwieldy. while R contains four basic data types, NumPy supports *far* more than this: for example, while R has a single integer type, NumPy supports *fourteen* basic integer types once you account for available precisions, signedness , and endianness of the encoding. Reserving a specific bit pattern in all vailable NumPy types would lead to an unwieldy amount of overhead in special-casing various operations for various types, likely even requiring a new fork of the NumPy package. Further, for the smaller data types(such as 8-bit integers), sacrificing a bit to use as a mask will significantly reduce the range of valeus it can represent.

NumPy does have support for masked arrays - that is, arrays that have a separate Boolean mask array attached for marking data as "good" or "bad". Pandas could have derived from this, but the overhead in both storage, computation, and cod emaintenance makes that an unattractive choice.

With these constraints in mind, Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values: the special floating-point `NaN`, and the Python `None` object. this choice has some side effects, as we will see, but in practice ends up being a good compromise in most cases of interests.

### `None`: Pythonic missing data

The first sentinel  value used by Pandas is `None`, a Python singleton object that is often used for missing data in Python code. Because it is a Python object, `None` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data tye `object`(i.e arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd 

In [3]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

this `dtype=object` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects. While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [6]:
for dtype in['object', 'int']:
    print(f"dtype = {dtype}")
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
68.3 ms ± 7.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
4.29 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` across an array with a `None` value, you will generally get an error:

In [7]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

This reflects the fact that addition between and integer and `None` is undefined

### `NaN`: Missing numerical data

The other missing data representation, `NaN`(acryonm for *Not a Number*) is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [9]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code. You should be aware that `NaN` is a bit like a data virus-it infects any other object it touches. Regardless of the operation, the result of arithmetic with `NaN` will be another `NaN`:

In [11]:
1 + np.nan

nan

In [13]:
0 * np.nan

nan

Note that this means that aggregates over the values are well defined(i.e they don't result in an error) but no always useful:

In [14]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy does provide some special aggregations that ignore these missing values:

In [15]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

Keep in min that `NaN` is specifically a floating-point value: there is no equivalent `NaN` value for integers, strings, or other types.

### NaN and None in Pandas

`NaN` and `None` both have their place, and Pandas is built to handle the two of them nearly interchangeably, convertin between them where appropriate:

In [16]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present. For example, if we set a value in an integer array to `np.nan`, it wil lautomatically be upcast to a floating-point type to accommodate the NA:

In [17]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [18]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the `None` to a `NaN` value. (Be aware that there is a proposal to add a nativ einteger NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

```
Typeclass 	Conversion When Storing NAs 	NA Sentinel Value
floating 	No change 	np.nan
object 	No change 	None or np.nan
integer 	Cast to float64 	np.nan
boolean 	Cast to object 	None or np.nan

```

Keep in mind that in Pandas, string data is always stored with an `object` dtype.